# Dir Structure
```
<your-repo>
|_ dataset
    |_ dataset.yaml
|_ yolov5*

```

In [39]:
# check if cuda is used
import torch
import os
from os import path
from PIL import Image
import matplotlib.pyplot as plt
import shutil

torch.cuda.is_available()

True

In [5]:
print(torch.version.cuda)

11.3


In [ ]:
!python -m torch.utils.collect_env

In [6]:
# create dataset.yaml file
if not path.exists('datasets'):
    os.mkdir('datasets')

In [28]:
from PIL import Image
import matplotlib.pyplot as plt

# preprocess raw images
def load_image(path):
    size = (1280, 720)
    img = Image.open(path).convert('RGB')
    return img.resize(size)

# crop image for standard 1280x720
def crop_face(img):
    size = img.size
    center = img.size[0]*0.5
    w = size[1]*0.2
    
    left = center-w
    right = center+w
    top = 75
    bottom = 500
    
    img = img.crop((left,top,right,bottom))
    
    return img

def preprocess():
    DIR = './datasets/training/raw'
    SAVE = './datasets/training/images'
    for file in os.listdir(DIR):
        img = load_image(path.join(DIR, file))
        img = crop_face(img)
        img.save(f"{SAVE}/{file.split('.')[0]}_body.jpg")

In [54]:
# check image health (missing labels, missing images, etc.)
def check_labels():
    DIR = 'datasets/training_small'
    results = []
    for file in os.listdir(path.join(DIR, 'labels')):
        name = file.split('.')[0] + '.jpg'
        for root, dirs, files in os.walk(path.join(DIR, 'images')):
            if name not in files:
                results.append(name)
                
    if len(results) == 0:
        return 'labels are healthy'
    return results

def check_images():
    DIR = 'datasets/training_small'
    results = []
    for file in os.listdir(path.join(DIR, 'images')):
        name = file.split('.')[0] + '.txt'
        for root, dirs, files in os.walk(path.join(DIR, 'labels')):
            if name not in files:
                results.append(file)
    return results

def move_files(files):
    source = 'datasets/training_small/images'
    dest = 'datasets/training_small'
    for file in files:
        full_path = path.join(source, file)
        shutil.move(full_path, dest)

print('Images without label')
x = check_images()
print(len(x))
x

Images without label
0


[]

# Template
`python yolov5/train.py --img 640 --batch 2 --epochs 1 --data datasets/training/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml`

In [ ]:
# training the model (remove mosaic)
python yolov5/train.py --img 640 --batch 2 --epochs 2 --data datasets/training/ak.yaml --weights yolov5/yolov5s.pt --hyp hyp.custom.yaml --rect

In [ ]:
# training on smaller dataset (experimental)
python yolov5/train.py --img 640 --batch 2 --epochs 200 --data datasets/training_small/ak.yaml --weights yolov5/yolov5m.pt --hyp hyp.custom.yaml --rect

In [35]:
# inference images need to have dimensions of 640x640 
def pre_inference():
    DIR = './datasets/inference/raw'
    SAVE = './datasets/inference/images'
    
    left = (0, 0, 640, 640)
    right = (640, 0, 1280, 640)
    crops = {'left': left, 'right': right}
    
    for file in os.listdir(DIR):
        img = load_image(path.join(DIR, file))
        
        for k, v in crops.items():
            proc = img.crop(v)
            proc.save(f"{SAVE}/{file.split('.')[0]}_{k}.jpg")
        
pre_inference()

In [22]:
import torch
model_path = 'yolov5/runs/train/medium_baseline_200/weights/best.onnx'
model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path, force_reload=True)



Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\aditi/.cache\torch\hub\master.zip
YOLOv5  2022-7-12 Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Loading yolov5\runs\train\medium_baseline_200\weights\best.onnx for ONNX Runtime inference...
C:\Users\aditi\miniconda3\envs\yolo\lib\site-packages\onnxruntime\capi\onnxruntime_inference_collection.py:55: UserWarning: Specified provider 'CUDAExecutionProvider' is not in available provider names.Available providers: 'CPUExecutionProvider'
  warnings.warn("Specified provider '{}' is not in available provider names."
Adding AutoShape... 


In [24]:
model.conf = 0.5
img = 'datasets/inference/images/oVcEpND_right.jpg'
result = model(img, size=640)

result.print()

result.xyxy

image 1/1: 640x640 2 hibiscuss
Speed: 8.1ms pre-process, 159.9ms inference, 3.8ms NMS per image at shape (1, 3, 640, 640)


[tensor([[ 99.17775, 223.11775, 203.21819, 362.58972,   0.92617,   7.00000],
         [302.93692, 221.52496, 401.61923, 367.02466,   0.90101,   7.00000]], device='cuda:0')]

In [2]:
!python yolov5/detect.py \
            --weights yolov5/runs/train/medium_baseline_200/weights/best.pt \
            --img 640 \
            --conf 0.5 \
            --source datasets/inference/images

detect: weights=['yolov5/runs/train/medium_baseline_200/weights/best.pt'], source=datasets/inference/images, data=yolov5\data\coco128.yaml, imgsz=[640, 640], conf_thres=0.5, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=False, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=False, augment=False, visualize=False, update=False, project=yolov5\runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False
YOLOv5  v6.1-289-g526e650 Python-3.9.12 torch-1.11.0+cu113 CUDA:0 (NVIDIA GeForce RTX 3060 Laptop GPU, 6144MiB)

Fusing layers... 
Model summary: 290 layers, 20913549 parameters, 0 gradients, 48.1 GFLOPs
image 1/29 D:\Repository\tracking-vision-yolo\datasets\inference\images\11629306773920_left.jpg: 640x640 Done. (0.018s)
image 2/29 D:\Repository\tracking-vision-yolo\datasets\inference\images\11629306773920_right.jpg: 640x640 1 midnight, 2 fangs, Done. (0.022s)
image 3/29 D:\Repository\tracki